# Lab: NumPy and Pandas

### 1. Pandas

* Read in file `life_expectancy_data.csv`

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("./life_expectancy_data.csv")

* Rename columns 

1. Make them all lower case, remove all hyphens, whitespace, and slashes.
2. Make "Income composition of resources" --> "composition", and "BMI" --> "avg_bmi".
3. Change the name of "under-five deaths" to something else of your choosing.

In [3]:
df.columns = df.columns.str.lower().str.replace(r'[\s\-\_\/]','', regex = True)
df = df.rename(columns={'incomecompositionofresources' : 'composition', 
                        'bmi' : 'avg_bmi',
                        'underfivedeaths' : 'under_five_deaths'})
df.columns

Index(['country', 'year', 'status', 'lifeexpectancy', 'adultmortality',
       'infantdeaths', 'alcohol', 'percentageexpenditure', 'hepatitisb',
       'measles', 'avg_bmi', 'under_five_deaths', 'polio', 'totalexpenditure',
       'diphtheria', 'hivaids', 'gdp', 'population', 'thinness119years',
       'thinness59years', 'composition', 'schooling'],
      dtype='object')

* Feature engineering

1. Combine the two "thinness" variables somehow
2. Transform the GDP and Population variables somehow
3. Make country and status variables into dummies. Give them a prefix of 'dum' to keep track of them
4. Combine GDP and one of the expenditure variables into a new variable

In [4]:
df['thinness_comb'] = df['thinness59years'] + df['thinness119years']
df['log_gdp'] = np.log2(df['gdp'])
df['log_population'] = np.log2(df['population'])
df = pd.get_dummies(df, columns=['country'], drop_first=True, prefix="dum")
df.head()
df['gdp_expenditure_ratio'] = df['gdp'] / df['totalexpenditure']

In [5]:
# to access all your dummy cols
df[[col for col in df if col.startswith('dum')]]

,dum_Albania,dum_Algeria,dum_Angola,dum_Antigua and Barbuda,dum_Argentina,dum_Armenia,dum_Australia,dum_Austria,dum_Azerbaijan,dum_Bahamas,...,dum_United Republic of Tanzania,dum_United States of America,dum_Uruguay,dum_Uzbekistan,dum_Vanuatu,dum_Venezuela (Bolivarian Republic of),dum_Viet Nam,dum_Yemen,dum_Zambia,dum_Zimbabwe
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2934,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2935,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2936,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


* Summarize data

1. How many observations are there for developing countries?
2. Drop all observations with missing values. How many rows were there before and how many are there now?
3. How old is the oldest observation?

In [6]:
print(df[df['status'] == 'Developing'].shape[0])
df = df.dropna()
print(df['year'].min())

2426
2000


* Generate outputs

1. Export the new dataset as a CSV in your lab folder.
2. Keep the following columns in a new dataset called `le2`: all your engineered features plus Life Expectancy, Schooling, GDP, total expenditures, and BMI. Drop everything else.

In [7]:
df.to_csv("le2.csv")

### 2. NumPy

* Define the following NumPy arrays:
    1. `X`, containing the `schooling` and `gdp` variables in `le2`
    2. `y`, containing only the life expectancy variable

In [8]:
X = np.array(df[['schooling','gdp']].fillna(0.0).astype(np.float64).to_numpy())
y = np.array(df[['lifeexpectancy']].fillna(0.0).astype(np.float64).to_numpy())
X = np.hstack((np.ones(X.shape[0]).reshape(X.shape[0],1),X))

* Regression by hand
    1. Using linear algebra operations, perform OLS to estimate $\beta = (X^TX)^{-1}X^TY$. What is the coefficient estimate for the effect of schooling on life expectancy?

In [9]:
beta = np.linalg.inv(X.T @ X) @ X.T @ y
beta

array([[4.33036289e+01],
       [2.09967715e+00],
       [9.89605585e-05]])

### 3. Random Forest

* Train a random forest classifier to predict whether a country is "developed" in a given year based on all the other features, including life expectancy.

In [10]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

# df['developed'] = 0
# df['developed'] = df.loc[df['status'] == 'Developed'] = 1
df['developed'] = df['status'].apply(lambda x: 1 if x == 'Developed' else 0)

X = df[['lifeexpectancy','adultmortality',
       'infantdeaths', 'alcohol', 'percentageexpenditure', 'hepatitisb',
       'measles', 'avg_bmi', 'under_five_deaths', 'polio', 'totalexpenditure',
       'diphtheria', 'hivaids', 'gdp', 'population', 'thinness119years',
       'thinness59years', 'composition', 'schooling']]
y = df['developed']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

rfc = RandomForestClassifier(random_state=42)

accuracies = cross_val_score(rfc, X_train, y_train, cv=5)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

print("Train Score:", np.mean(accuracies))
print("Test Score:", rfc.score(X_test, y_test))


Train Score: 0.9833275723009564
Test Score: 0.9696969696969697


* **Bonus:** Can you figure out how to extract feature importance from the classifier?

In [11]:
feature_importance = np.array(rfc.feature_importances_)
feature_names = np.array(X.columns)

data = pd.DataFrame({'feature_names': feature_names, 'feature_importance': feature_importance})